In [ ]:
#@title # **نصب**
#@markdown ####حدود دو دقیقه زمان میبره
from IPython.display import clear_output
import subprocess
import os

colab_path = "/content"
kaggle_path = "/kaggle/working"

if os.path.exists(colab_path):
  print("Welcome to Colab Notebook")
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  path = "/content"
elif os.path.exists(kaggle_path):
  print("Welcome to Kaggle Notebook")
  path = "/kaggle/working"

!pip install "audio-separator[gpu]==0.17.5"
subprocess.run(["pip", "install", "demucs"])
!pip install aria2
!pip install yt_dlp
!mkdir models
!mkdir temp
!aria2c https://huggingface.co/Eddycrack864/Drumsep/resolve/main/modelo_final.th -o models/drumsep.th
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/
!pip install noisereduce librosa soundfile ipywidgetsclear_output()
print('Installation done !')

In [ ]:
#@title #**جداسازی صدا از موزیک**
import os
import glob
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, extensions, output_format, model):
    found_files = []

    dictmodel = {
        'BS-Roformer-Viperx-1297.ckpt': 'model_bs_roformer_ep_317_sdr_12.9755.ckpt',
        'BS-Roformer-Viperx-1296.ckpt': 'model_bs_roformer_ep_368_sdr_12.9628.ckpt',
        'BS-Roformer-Viperx-1053.ckpt': 'model_bs_roformer_ep_937_sdr_10.5309.ckpt',
        'Mel-Roformer-Viperx-1143.ckpt': 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt'
    }
    roformer_model = dictmodel[model]

    if checker(audio_input):
        downloader(audio_input)
        audio_input = f"{path}/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            prompt = f'audio-separator "{file_path}" --model_filename {roformer_model} --output_dir={output_folder} --output_format={output_format} --normalization=0.9 --mdxc_overlap={overlap} --mdxc_segment_size={segment_size}'
            !$prompt

    if audio_input == f"{path}/temp":
        temp_files = glob.glob(f"{path}/temp/*")
        for file in temp_files:
            os.remove(file)

#@markdown لینک آهنگ مورد نظر را وارد کنید
audio_input = "" #@param {type:"string"}
#@markdown ####لیست سایت هایی که لینکشون پشتیبانی میشه >>> [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/Vocales" #@param {type:"string"}
#@markdown Select the model:
model = "BS-Roformer-Viperx-1297.ckpt" #@param ["BS-Roformer-Viperx-1297.ckpt", "BS-Roformer-Viperx-1296.ckpt", "BS-Roformer-Viperx-1053.ckpt", "Mel-Roformer-Viperx-1143.ckpt"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Amount of overlap between prediction windows.
overlap = 4 #@param {type:"slider", min:2, max:4, step:1}
#@markdown Larger consumes more resources, but may give better results.
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(audio_input, output_folder, extensions, output_format, model)

In [ ]:

import os
import librosa
import soundfile as sf
import numpy as np
from pydub import AudioSegment
from pydub.silence import split_on_silence

#@title 🎵 ترکیب و پاکسازی فایل‌های صوتی
output_path = "final_vocals.wav" #@param {type:"string"}

def process_vocals():
    # مسیر پوشه حاوی فایل‌ها
    folder_path = "/content/drive/MyDrive/Vocales"

    print("🔍 جستجوی فایل‌های صوتی...")
    # پیدا کردن تمام فایل‌های صوتی که شامل Vocals هستند
    vocal_files = [f for f in os.listdir(folder_path) if 'Vocals' in f]

    if not vocal_files:
        print("❌ هیچ فایل صوتی با نام Vocals پیدا نشد!")
        return

    print(f"✅ {len(vocal_files)} فایل صوتی پیدا شد")

    # ترکیب فایل‌ها
    combined_audio = None
    for file in vocal_files:
        print(f"📝 پردازش فایل: {file}")
        file_path = os.path.join(folder_path, file)
        audio = AudioSegment.from_file(file_path)

        if combined_audio is None:
            combined_audio = audio
        else:
            combined_audio += audio

    print("✂️ حذف سکوت‌ها...")
    # حذف سکوت‌ها
    chunks = split_on_silence(
        combined_audio,
        min_silence_len=1000,  # حداقل طول سکوت (میلی‌ثانیه)
        silence_thresh=-40,    # آستانه تشخیص سکوت
        keep_silence=100       # حفظ کمی سکوت برای طبیعی بودن
    )

    # ترکیب قطعات بدون سکوت
    print("🎵 ترکیب نهایی قطعات...")
    final_audio = chunks[0]
    for chunk in chunks[1:]:
        final_audio += chunk

    # ذخیره فایل نهایی
    final_audio.export(output_path, format="wav")
    print(f"💾 فایل نهایی در {output_path} ذخیره شد")

    return output_path

try:
    result = process_vocals()
    if result:
        print("✨ عملیات با موفقیت انجام شد!")
except Exception as e:
    print(f"❌ خطا: {str(e)}")

In [ ]:

import librosa
import soundfile as sf
import noisereduce as nr
import numpy as np
from scipy.signal import butter, filtfilt
from IPython.display import Audio, display

#@title 🎙️ پردازش صدا با حذف اکو
input_path = "/content/final_vocals.wav" #@param {type:"string"}
output_path = "clear_natural_voice.wav" #@param {type:"string"}
echo_reduction = 0.9 #@param {type:"slider", min:0.7, max:0.95, step:0.05}
presence = 0.1 #@param {type:"slider", min:0.1, max:0.3, step:0.05}

def enhanced_voice_processor(input_path, output_path, echo_reduction, presence):
    print("🎙️ شروع پردازش...")
    audio, sr = librosa.load(input_path, sr=44100, mono=True)

    # حذف اکو قوی
    print("🔊 حذف اکو...")
    echo_reduced = nr.reduce_noise(
        y=audio,
        sr=sr,
        prop_decrease=echo_reduction,
        stationary=False,
        n_fft=2048,
        win_length=2048,
        n_std_thresh_stationary=1.2
    )

    # تنظیم فضای باز
    print("🌳 تنظیم فضای طبیعی...")
    b1, a1 = butter(2, [200/22050, 8000/22050], btype='band')
    b2, a2 = butter(2, 4000/22050, btype='high')

    filtered = filtfilt(b1, a1, echo_reduced)
    high_freq = filtfilt(b2, a2, echo_reduced) * 0.2
    enhanced = filtered + (high_freq * presence)

    # نرمالایز نهایی
    final_audio = librosa.util.normalize(enhanced) * 0.95

    sf.write(output_path, final_audio, sr, 'PCM_24')
    print("✨ پردازش با موفقیت انجام شد!")

    return final_audio, sr

if input_path:
    try:
        result, sr = enhanced_voice_processor(input_path, output_path, echo_reduction, presence)
        display(Audio(output_path))
    except Exception as e:
        print(f"❌ خطا: {str(e)}")
else:
    print("⚠️ لطفا مسیر فایل صوتی را وارد کنید")

In [ ]:
#@title 🗑️ حذف تمام صداها از درایو
#@markdown #### با اجرای این سلول تمام صداها از پوشه وکالز حذف میشوند
import os
import shutil

def clean_vocals_folder():
    folder_path = "/content/drive/MyDrive/Vocales"

    print("🔍 در حال بررسی پوشه...")
    if os.path.exists(folder_path):
        # حذف تمام فایل‌ها در پوشه
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"❌ خطا در حذف {file}: {e}")
        print("✨ تمام فایل‌ها با موفقیت حذف شدند!")
    else:
        print("⚠️ پوشه مورد نظر یافت نشد!")

clean_vocals_folder()
